##### EDA

In [2]:
import torchxrayvision as xrv

C:\Program Files\Python312\Lib\site-packages\torchxrayvision\utils.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
d = xrv.datasets.COVID19_Dataset(imgpath="data/images/", csvpath="data/csv/metadata.csv")

In [4]:
img = d[0]['img'][0]
print(f"sample image of shape {img.shape}")
print(f"dataset of length {len(d)}")
print(img)

sample image of shape (888, 882)
dataset of length 535
[[1024.      1015.9686  1007.93726 ... -469.83527 -477.86664 -477.86664]
 [1024.      1024.      1015.9686  ... -493.92938 -493.92938 -493.92938]
 [1024.      1024.      1015.9686  ... -501.96075 -509.99213 -509.99213]
 ...
 [1024.      1024.       983.84314 ... -172.6745  -156.61176 -148.58038]
 [1024.      1024.       983.84314 ... -252.98822 -244.95685 -244.95685]
 [1024.      1024.       983.84314 ... -437.70978 -429.6784  -421.64703]]


##### Dataloader

In [1]:
from dataLoader import *
from torch.utils.data import DataLoader

C:\Users\Hoo Kai Sng\OneDrive - National University of Singapore\Documents\NUS\CS3244\Project\xrayenv\Lib\site-packages\torchxrayvision\utils.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
train_dataset = Covid19DataSet('train', transform = 'vanilla')
augmented_dataset = Covid19DataSet('train', transform = 'augment')
validation_dataset = Covid19DataSet('val', transform = 'vanilla')
test_dataset = Covid19DataSet('test', transform = 'vanilla')

In [3]:
BATCH_SIZE = 16
train_loader = DataLoader(dataset = train_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 4)
validation_loader = DataLoader(dataset = validation_dataset, batch_size = BATCH_SIZE, shuffle = False, num_workers = 4)
test_loader = DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE, shuffle = False, num_workers = 4)

##### Building of model

In [4]:
from models import *

In [5]:
model_11 = ConvNet()

In [6]:
model_12 = ConvNet()

In [7]:
model_21 = ConvNetGlobPooling()

In [8]:
model_22 = ConvNetGlobPooling()

##### Training Model

In [9]:
from evaluation import *

In [18]:
import torch
from torch.optim import lr_scheduler
import torch.nn as nn
import time
import copy

# Need to Re consider validation and stepping of gradients
def train_model(model, train_loader, validation_loader, epochs = 5, learning_rate = 0.01, grad_criterion = nn.BCEWithLogitsLoss()):
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min', factor = 0.7, patience = 1)
    best_model, best_met = copy.deepcopy(model.state_dict()), 0.0
    starttime = time.time()
    
    print("Training model...")
    model.train()

    for epoch in range(epochs):
        for phase in ['train', 'val']:
            epoch_loss = tp = fp = fn = n = 0.0
            if phase == 'train':
                model.train()
            else:
                model.eval()  
            target_loader = train_loader if phase == 'train' else validation_loader
            for i, (images, labels) in enumerate(target_loader):

                with torch.set_grad_enabled(phase == 'train'):
                    # forward pass
                    y_preds = model(images)
                    pred_labels = (torch.sigmoid(y_preds) >= 0.5).float()
                    loss = grad_criterion(y_preds, labels)
                    epoch_loss += loss.item()

                    if phase == 'train':
                        # backward prop
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()    

                for i in range(len(labels)):
                    pred_val, label_val = pred_labels[i].item(), labels[i].item() 
                    tp += 1 if (pred_val == label_val == 1 ) else 0.0
                    fp += 1 if (pred_val == 1 and label_val == 0) else 0.0
                    fn += 1 if (pred_val == 0 and label_val == 1) else 0.0
                    n += 1
                    
            epoch_met = (2 * tp) / (2 * tp + fp + fn) 

            if phase == 'train':
                # scheduler step
                scheduler.step(epoch_loss)

            if phase == 'val' and epoch_met > best_met:            
                # validate model
                best_met = epoch_met
                best_model = copy.deepcopy(model.state_dict())
                
        print(f"Epoch [{epoch+1}/{epochs}] | Epoch Metric: {epoch_met:.4f} | Epoch Loss: {epoch_loss:.4f}")
    
    elapsedtime = time.time() - starttime
    mins, sec = elapsedtime//60, elapsedtime%60
    print(f"Training completed in {mins:.0f}mins {sec:.0f}s")
    print(f"Best validation: {best_met}")

    model.load_state_dict(best_model)
    return model
    

def evaluate_model(model, test_loader, criteria):
    with torch.no_grad():
        tp = tn = fp = fn = total = 0
        for image, label in test_loader:
            output = model(image)

            y_pred = (torch.sigmoid(output) >= 0.5).float()
            n = len(y_pred)
            for i in range(n):
                pred_val, label_val = y_pred[i].item(), label[i].item() 
                tp += (pred_val == label_val == 1)
                tn += (pred_val == label_val == 0)
                fp += (pred_val == 1 and label_val == 0)
                fn += (pred_val == 0 and label_val == 1)
                total += 1
    match criteria:
        case 'acc':
            return (tp + tn) / total
        case 'rec':
            return tp / (tp + fn)
        case 'prec':
            return tp / (tp + fp)
        case 'f1':
            return (2 * tp) / (2 * tp + fp + fn) 


In [19]:
# Training for a default of 5 epochs

In [20]:
train_model(model_21, train_loader, validation_loader)

Training model...
Epoch [1/5] | Epoch Metric: 0.8737 | Epoch Loss: 3.7926
Epoch [2/5] | Epoch Metric: 0.8737 | Epoch Loss: 3.9400
Epoch [3/5] | Epoch Metric: 0.8737 | Epoch Loss: 3.9095
Epoch [4/5] | Epoch Metric: 0.8737 | Epoch Loss: 3.8965
Epoch [5/5] | Epoch Metric: 0.8737 | Epoch Loss: 3.7797
Training completed in 5mins 55s
Best validation: 0.8736842105263158


ConvNetGlobPooling(
  (conv1): Conv2d(1, 8, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1))
  (global_avg_pool): AdaptiveAvgPool2d(output_size=1)
  (fc1): Linear(in_features=16, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
)

In [21]:
train_model(model_22, train_loader, validation_loader)

Training model...
Epoch [1/5] | Epoch Metric: 0.8737 | Epoch Loss: 3.7955
Epoch [2/5] | Epoch Metric: 0.8737 | Epoch Loss: 3.7870


KeyboardInterrupt: 

##### Evaluation

In [ ]:
metric_21_prec = evaluate_model(model_21, test_loader, 'prec') * 100
metric_21_rec = evaluate_model(model_21, test_loader, 'rec') * 100
print(f"model 2.1: Precision = {metric_21_prec:.0f}% | Recall = {metric_21_rec:.0f}%")

In [ ]:
metric_22_prec = evaluate_model(model_21, test_loader, 'prec') * 100
metric_22_rec = evaluate_model(model_22, test_loader, 'rec') * 100
print(f"model 2.2: Precision = {metric_21_prec:.0f}% | Recall = {metric_21_rec:.0f}%")

##### Visualisation